# Analysis

### 1. Loading the project

In [2]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

12:14:33+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [3]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

### 2. Database overview

In [4]:
bd.databases

Databases dictionary with 3 object(s):
	bw25_db
	ecoinvent-3.10-biosphere
	ecoinvent310clca

In [5]:
bd.projects.current

'brightway25'

In [6]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
ceramics_db = bd.Database('bw25_db')

### 3. Database exploration

In [7]:
ceramics_db.search('ceramics')

['ceramics production' (kilogram, GLO, None)]

In [8]:
ei_clca.search('ceramics')

['market for sanitary ceramics' (kilogram, GLO, None),
 'sanitary ceramics production' (kilogram, RoW, None),
 'sanitary ceramics production' (kilogram, CH, None),
 'refractory production, basic, packed' (kilogram, RoW, None),
 'refractory production, fireclay, packed' (kilogram, DE, None),
 'refractory production, fireclay, packed' (kilogram, RoW, None),
 'refractory production, basic, packed' (kilogram, DE, None),
 'market for ceramic tile' (kilogram, GLO, None),
 'ceramic tile production' (kilogram, CH, None),
 'market for feldspar' (kilogram, GLO, None),
 'power saw production, with catalytic converter' (unit, GLO, None),
 'ceramic tile production' (kilogram, RoW, None),
 'market for titanium dioxide' (kilogram, RoW, None),
 'market for titanium dioxide' (kilogram, RER, None),
 'market for kaolin' (kilogram, GLO, None),
 'strontium carbonate production' (kilogram, GLO, None),
 'sodium tripolyphosphate production' (kilogram, RoW, None),
 'sodium tripolyphosphate production' (kilogra

In [15]:
# define a process as object in the project
ceramics = ceramics_db.get(name = 'ceramics production', location = "GLO", unit = 'kilogram')


In [9]:
ceramics_ei = ei_clca.get(name = 'market for sanitary ceramics', location = "GLO", unit = 'kilogram')

### 4. Impact assessment methods

In [10]:
lcia_gwp100 = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
lcia_water = ('EF v3.1','water use','user deprivation potential (deprivation-weighted water consumption)')
lcia_land = ('EF v3.1', 'land use', 'soil quality index')

### 4. LCI 

In [12]:
# export the LCI to e
bi.export.excel.write_lci_excel(ei_clca.name,
                                objs=[ceramics_ei],
                                dirpath=Path.cwd())

'c:\\Users\\TimWeber\\repos_20LCA\\brightway25\\lci-ecoinvent310clca.xlsx'

### 5. LCIA

In [16]:
# Quick LCIA calculation
ceramics_lca = ceramics.lca(lcia_gwp100)
ceramics_lca.score


c:\Users\TimWeber\anaconda3\envs\brightway25\Lib\site-packages\bw2calc\lca_base.py:87: UserWarning: No valid biosphere flows found. No inventory results can be calculated, `lcia` will raise an error
  warnings.warn(


EmptyBiosphere: 

### 6. Contribution analysis

In [ ]:
# One option to do a contribution analysis is to use the recursive calculation method
bwa.print_recursive_calculation(ceramics,
lcia_method=lcia_gwp100,max_level=3,cutoff=0.05)

In [ ]:
# Another option is to use the recursive calculation to an object, which returns a DataFrame
ceramics_ca_gwp = bwa.utils.recursive_calculation_to_object(ceramics,
                                          lcia_method=lcia_gwp100,
                                          max_level=1,
                                          cutoff=0.02,
                                          as_dataframe=True,
                                          )
ceramics_ca_gwp

In [ ]:
# Elementary flows contribution analysis by process
ceramics_lca.to_dataframe().pivot_table(index=['col_name', 'row_name'], values='amount', aggfunc='sum').sort_values(by='amount', ascending=False)

In [ ]:
# Elementary flows contribution analysis as summary table
ceramics_lca.to_dataframe().pivot_table(index='row_name',values='amount',aggfunc='sum')

### 7. Plot Contribution analysis graphs

In [ ]:
# Filter out the parent processes that are not relevant for the contribution analysis
ceramics_ca_gwp = ceramics_ca_gwp.dropna(subset='parent') 



In [ ]:
f, ax = plt.subplots(figsize=(6, 3))
sns.barplot(y='name', x='score', data=ceramics_ca_gwp, ax=ax)
ax.set(
    title='Contribution analysis of smartphone production',
    xlabel='CO2 eq. emissions (kg)',
    ylabel='')
plt.tight_layout()
plt.show()
